In [ ]:
import numpy as np
import shapely.geometry as shg
import matplotlib.pyplot as pyplot
import copy as cp
import scipy.spatial.distance as scd


%matplotlib inline

COLOR = {
    True:  '#6699cc',
    False: '#ffcc33'
    }

def v_color(ob):
    return COLOR[ob.is_simple]

def plot_coords(ax, ob):
    for line in ob:
        x, y = line.xy
        ax.plot(x, y, 'o', color='#999999', zorder=1)

def plot_bounds(ax, ob):
    x, y = zip(*list((p.x, p.y) for p in ob.boundary))
    ax.plot(x, y, 'o', color='#000000', zorder=1)

def plot_lines(ax, ob):
    for line in ob:
        x, y = line.xy
        ax.plot(x, y, color=v_color(ob), alpha=0.7, linewidth=3, solid_capstyle='round', zorder=2)

#no of interations
iterations = 100

free_ride = 50

#max noise angle component
max = 30
# angs 

angs = np.linspace(0,360,40)[:-1]

parent_id = 1
inter_pts = 20

max_exp = 2

min_dist = .6
r_pts = np.linspace(1.,min_dist + .1,inter_pts)
bfactor = 1

#stack initial point

segments = []
segments_term = [np.array([0.,0])]

#child to parent mapping

chp = {}

#parent to child mapping

pch = {}

#node angle

pang = {}

pang[0] = 0

tree = shg.MultiLineString(segments)

#nodes

node_idx = 0

def branch(rot,r=r_pts):
    start_pt = np.array([1., 0])
    
    r = bfactor*np.random.choice(r)
    
    R = r*np.array([
                    [ np.cos(rot*180./np.pi), -np.sin(rot*180./np.pi)],
                    [ np.sin(rot*180./np.pi), np.cos(rot*180./np.pi)]
                ])
    return np.dot(R,start_pt)



for k in np.arange(iterations):
    
    #ang = (np.random.random()*(-2*max)+max) + prev_ang
    #new_segt = segments_term[node-1], (segments_term[node-1] + branch(ang))
    #delayed_tree = shg.MultiLineString(segments[:-1])

    #test_line = shg.LineString(new_segt)
    #dist = delayed_tree.distance(test_line)
    
    if k < 2:
        
        ang = (np.random.random()*(-2*max)+max)
        #r = np.max(bfactor*np.random.random(), min_dist+.1)
        
        new_segt = segments_term[node_idx], (segments_term[node_idx] + branch(ang))

        #initial node
        
        prev_ang = ang
        segments.append( shg.LineString(cp.copy(new_segt)) )
        segments_term.append(new_segt[1])
        #tree = shg.MultiLineString(segments)
        
        node_idx += 1
        
        pch[node_idx-1] = (node_idx,)
        chp[node_idx] = node_idx-1
        pang[node_idx] = ang
        
        
    elif k < free_ride:
        counter = 0
        hammerTime = True
        delayed_tree = shg.MultiLineString(segments[:-1])
        ang = pang[ node_idx ] + np.random.random()*(-1*max)+max
        #while counter < 10 and hammerTime:
        for ang in ang:
            
            #ang = (np.random.random()*(-2*max)+2*max)
            
            new_segt = segments_term[-1], (segments_term[-1] + branch(ang))
            test_line = shg.LineString(new_segt)
            dist = delayed_tree.distance(test_line)
                    
            if dist > min_dist:
                        
                segments.append( shg.LineString(cp.copy(new_segt)) )
                segments_term.append(new_segt[1])
                        
                #increment node number
                node_idx += 1

                #update dictionaries
                chp[ node_idx ] = node_idx-1
                pch[ node_idx-1 ] = (node_idx,)
                pang[ node_idx ] = ang
                hammerTime = False
               
                
                #break
                        
            counter += 1
        
    else:
        
        #choose node to expand at random
        nodes_to_exp = np.random.choice(node_idx-1,size=node_idx-1,replace=False)
        for node in nodes_to_exp:

            #get parent node 
            
            try:
                parent_id = chp[node]
            except:
                'not a child'
                
            #get child nodes
            
            try:
                children = pch[node]
            except:
                'not a parent'
            
            
            not_neighbours = [ segments[idx] for idx in np.arange(node_idx) if idx not in (children + (parent_id,))]
            
            delayed_tree = shg.MultiLineString(not_neighbours)
            counter = 0
            hammerTime = True
            
            while counter < 10 and hammerTime:
                ang = pang[parent_id] + np.random.random()*(-2*max)+2*max
            
            
            #for ang in noise_ang:
            
                #ang = (np.random.random()*(-2*max)+2*max)
                
                new_segt = segments_term[parent_id], (segments_term[parent_id] + branch(ang))
                test_line = shg.LineString(new_segt)
                dist = delayed_tree.distance(test_line)
                
                if dist > min_dist:
                        
                    segments.append( shg.LineString(cp.copy(new_segt)) )
                    segments_term.append(new_segt[1])
                        
                    #increment node number
                    node_idx += 1
                    #update dictionaries
                    chp[ node_idx ] = parent_id
                    pch[ parent_id ] = pch[ parent_id ] + (node_idx,)
                    pang[ node_idx ] = ang                                           
                    #break
                    
                    hammerTime = False
                        
                counter += 1
            


tree = shg.MultiLineString(segments)  
    
print tree.length


fig = pyplot.figure(figsize=(12,12), dpi=300)

ax = fig.add_subplot(111)
plot_coords(ax, tree)
plot_bounds(ax, tree)
plot_lines(ax, tree)


print node_idx , len(segments), '\n **********'
